In [1]:
import pandas as pd
import re

In [2]:
from gensim.test.utils import datapath
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import gensim.corpora as corpora


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lucia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from pprint import pprint


In [74]:
df=pd.read_csv("english_full.csv")

In [11]:
#df=pd.read_csv("semantic.csv")

In [12]:
def nonum(s):
    pattern = r'[0-9]'
    return re.sub(pattern, '', s)



In [13]:
df=df.dropna(subset=["text"])

In [14]:
df["text"]=df["text"].apply(nonum)

In [15]:
len(df["text"])

3016

Convert articles to bag of words

In [61]:

stop_words = stopwords.words('english')
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = list(df["text"])
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

['contents', 'list', 'available', 'vilnius', 'university', 'press', 'received', 'accepted', 'copyright', 'jukka', 'tyrkk', 'ilkka', 'mkinen', 'published', 'vilnius', 'university', 'press', 'open', 'access', 'journal', 'distributed', 'terms', 'creative', 'commons', 'attribution', 'licence', 'permits', 'unrestricted', 'use', 'distribution']


In [62]:
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 2), (1, 3), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1), (7, 2), (8, 1), (9, 12), (10, 3), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 2), (17, 2), (18, 1), (19, 1), (20, 11), (21, 1), (22, 1), (23, 1), (24, 2), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1)]


In [63]:
# number of topics
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaModel(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.006*"de" + 0.003*"et" + 0.003*"al" + 0.003*"one" + 0.003*"study" + '
  '0.002*"data" + 0.002*"health" + 0.002*"yang" + 0.002*"research" + '
  '0.002*"dan"'),
 (1,
  '0.010*"learning" + 0.009*"students" + 0.006*"social" + 0.006*"use" + '
  '0.005*"research" + 0.005*"media" + 0.004*"online" + 0.004*"google" + '
  '0.004*"study" + 0.003*"using"'),
 (2,
  '0.007*"social" + 0.006*"media" + 0.004*"research" + 0.004*"information" + '
  '0.004*"al" + 0.004*"google" + 0.004*"study" + 0.004*"et" + 0.003*"also" + '
  '0.003*"one"'),
 (3,
  '0.008*"data" + 0.004*"information" + 0.004*"social" + 0.004*"et" + '
  '0.004*"users" + 0.004*"al" + 0.004*"use" + 0.003*"using" + 0.003*"google" + '
  '0.003*"also"'),
 (4,
  '0.009*"die" + 0.004*"et" + 0.004*"data" + 0.003*"di" + 0.003*"van" + '
  '0.003*"al" + 0.003*"rice" + 0.003*"en" + 0.003*"teks" + 0.002*"time"'),
 (5,
  '0.011*"dan" + 0.011*"yang" + 0.005*"dengan" + 0.005*"google" + '
  '0.005*"dalam" + 0.005*"data" + 0.004*"pada" + 0.004*"un

In [65]:
#reshape topics object
topic_dict = {i: [] for i in range(10)}  

#Loop over all the documents to group the probability of each topic

for docID in range(len(df["text"])):
    topic_vector = lda_model[corpus[docID]]
    for topicID, prob in topic_vector:
        topic_dict[topicID].append((docID, prob))

In [49]:
#print articles with high topic prevalence
sorted(
    topic_dict[3], reverse=True,
    key=lambda x: x[1]
)

[(1800, 0.9893825),
 (1697, 0.95316964),
 (1398, 0.8477653),
 (2028, 0.8232189),
 (1397, 0.8142416),
 (1589, 0.8067991),
 (1884, 0.7982115),
 (1170, 0.77538985),
 (1007, 0.7086536),
 (689, 0.7049754),
 (1709, 0.61674047),
 (1378, 0.61365795),
 (549, 0.60591084),
 (857, 0.54758465),
 (691, 0.5470126),
 (1388, 0.5338859),
 (1282, 0.53231025),
 (1225, 0.52507806),
 (1994, 0.524302),
 (1581, 0.49330178),
 (519, 0.4744339),
 (719, 0.46843275),
 (1612, 0.46059307),
 (1900, 0.44996136),
 (1008, 0.44282386),
 (845, 0.43763015),
 (813, 0.43191242),
 (820, 0.42871398),
 (1768, 0.42240146),
 (715, 0.4095569),
 (1281, 0.40220392),
 (1221, 0.4012269),
 (547, 0.40053913),
 (961, 0.39655682),
 (1624, 0.3880971),
 (1226, 0.371495),
 (1591, 0.36824852),
 (540, 0.36233485),
 (1587, 0.36094952),
 (585, 0.35486415),
 (713, 0.35474047),
 (842, 0.3494429),
 (539, 0.34896663),
 (1541, 0.3485051),
 (698, 0.34825686),
 (843, 0.3453238),
 (853, 0.34516385),
 (532, 0.34441504),
 (546, 0.34152856),
 (703, 0.33779

In [60]:
df=df.reset_index()

In [53]:
df["title"][1800]

'the earthquakes on 12 april 1998 and 12 july 2004 in krn mountains nw slovenia  time distribution of aftershocks and magnitudefrequency relation'

In [66]:
#remove articles with high topic prevalence
rem=[]
for i in topic_dict[4]:
    if i[1]>0.85:
        rem.append(i[0])

In [67]:
for i in topic_dict[5]:
    if i[1]>0.9:
        rem.append(i[0])

In [ ]:
for i in topic_dict[16]:
    if i[1]>0.95:
        rem.append(i[0])

In [69]:
df=df.drop(index=rem)

In [71]:
df.to_csv("semantic_full.csv")

In [54]:
pprint(lda_model.print_topics())


[(0,
  '0.006*"data" + 0.004*"google" + 0.003*"use" + 0.003*"et" + 0.003*"social" + '
  '0.003*"information" + 0.003*"search" + 0.003*"also" + 0.003*"time" + '
  '0.003*"new"'),
 (1,
  '0.005*"et" + 0.005*"al" + 0.005*"social" + 0.004*"media" + 0.004*"data" + '
  '0.003*"information" + 0.003*"one" + 0.003*"use" + 0.003*"used" + '
  '0.003*"new"'),
 (2,
  '0.004*"de" + 0.004*"die" + 0.004*"one" + 0.004*"university" + 0.003*"new" + '
  '0.002*"et" + 0.002*"al" + 0.002*"en" + 0.002*"would" + 0.002*"also"'),
 (3,
  '0.018*"de" + 0.008*"la" + 0.006*"en" + 0.006*"el" + 0.006*"los" + '
  '0.004*"las" + 0.004*"que" + 0.004*"se" + 0.004*"al" + 0.003*"et"'),
 (4,
  '0.065*"de" + 0.036*"la" + 0.025*"en" + 0.019*"que" + 0.018*"el" + '
  '0.012*"los" + 0.010*"se" + 0.009*"del" + 0.008*"las" + 0.008*"un"'),
 (5,
  '0.005*"data" + 0.005*"web" + 0.004*"research" + 0.004*"information" + '
  '0.004*"digital" + 0.003*"use" + 0.003*"social" + 0.002*"al" + 0.002*"de" + '
  '0.002*"technology"'),
 (6,
  '0.

Visualization

In [64]:
import pyLDAvis
import pyLDAvis.gensim_models
vis = pyLDAvis.gensim_models.prepare(topic_model=lda_model, 
                              corpus=corpus, 
                              dictionary=id2word)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

C:\Users\lucia\miniconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [21]:
pyLDAvis.save_html(vis, 'lda.html')